In [35]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
    builder. \
    appName("SparkDemo"). \
    config('spark.ui.port','0'). \
    config('spark.shuffle.useOldFethProtocol','true'). \
    config('spark.sql.warehouse.dir',f'/user/{username}/warehouse'). \
    enableHiveSupport(). \
    master('yarn'). \
    getOrCreate()

In [36]:
spark.sql(" select member_id, count(*) as total from itv009959_lending_club.customers group by member_id order by total desc")

member_id,total
e4c167053d5418230...,5
3f87585a20f702838...,4
76b577467eda5bdbc...,4
ad8e5d384dae17e06...,4
066ddaa64bee66dff...,3
3ae415acd6bbfaac1...,3
d9ce4046daf599732...,3
059d401bb603d9a80...,3
a2356725aa9da92c6...,3
c563428cb58da48ff...,3


In [3]:
spark.sql(" select * from itv009959_lending_club.customers where member_id like 'e4c167053d5418230%'")

member_id,emp_title,emp_length,home_ownership,annual_income,address_state,address_zipcode,address_country,grade,sub_grade,verification_status,total_high_credit_limit,application_type,join_annual_income,verification_status_joint,ingest_date
e4c167053d5418230...,null,6,MORTGAGE,55000.0,IL,604xx,USA,B,B5,Verified,207300.0,Individual,null,null,2024-03-12 17:37:...
e4c167053d5418230...,null,6,MORTGAGE,55000.0,IL,604xx,USA,B,B5,Verified,171165.0,Individual,null,null,2024-03-12 17:37:...
e4c167053d5418230...,null,6,MORTGAGE,55000.0,IL,604xx,USA,B,B5,Verified,138780.0,Individual,null,null,2024-03-12 17:37:...
e4c167053d5418230...,null,6,MORTGAGE,55000.0,IL,604xx,USA,B,B5,Verified,129833.0,Individual,null,null,2024-03-12 17:37:...
e4c167053d5418230...,null,6,MORTGAGE,55000.0,IL,604xx,USA,B,B5,Verified,110907.0,Individual,null,null,2024-03-12 17:37:...


In [37]:
bad_data_customers = spark.sql(" select member_id from (select member_id, count(*) as total from itv009959_lending_club.customers group by member_id having total>1)")

In [3]:
bad_data_customers

member_id
eebbd89aa7efc13e7...
5130d0087970e032e...
2bae2e4dd6d5f2b21...
a2af7506825a7dcff...
fc0e468bff11ac7c3...
a53e2f488d2d76a30...
761b2f1e61999e14e...
7115ace193e13d8f3...
3b8ffe19f24749a73...
4231a55d0199c619a...


In [7]:
bad_data_customers.count()

3157

In [38]:
bad_data_loans_defaulters_delinq = spark.sql("""
select member_id from 
(select member_id, count(*) as total 
from itv009959_lending_club.loans_defaulters_delinq 
group by member_id having total>1) 
""")

In [11]:
bad_data_loans_defaulters_delinq

member_id
8d2c605c7ad9209cc...
bbe43331566910d55...
6918b9861ba5a4c67...
b5ded5638e54e1692...
a93a44c9d83793451...
6c2b63ff231e520d4...
694c6cb86608015e9...
58dde656f747cee8b...
0b44a10bc749eabb2...
77db5fdf951dd04b2...


In [39]:
bad_data_loans_defaulters_detail_rec_enq_df = spark.sql("""
select member_id from 
(select member_id, count(*) as total 
from itv009959_lending_club.loans_defaulters_delinq_detail_records
group by member_id having total>1) 
""")

In [13]:
bad_data_loans_defaulters_detail_rec_enq_df

member_id
cbede54df344cdb94...
cd1fdca829c443fa7...
761b2f1e61999e14e...
3b8ffe19f24749a73...
2bae2e4dd6d5f2b21...
7115ace193e13d8f3...
f284044b881f218c0...
eebbd89aa7efc13e7...
01b2223757c3b62e7...
a53e2f488d2d76a30...


In [14]:
bad_data_loans_defaulters_delinq.count()

144

In [15]:
bad_data_loans_defaulters_detail_rec_enq_df.count()

3189

In [21]:
bad_data_customers.repartition(1).write \
.format("csv") \
.option("header",True) \
.mode('overwrite') \
.option("path","/user/itv009959/lendingclubproject/bad/bad_data_customers_csv") \
.save()

In [8]:
bad_data_loans_defaulters_delinq.repartition(1).write \
.format("csv") \
.option("header",True) \
.mode('overwrite') \
.option("path","/user/itv009959/lendingclubproject/bad/bad_data_loans_defaulters_delinq_csv") \
.save()

In [9]:
bad_data_loans_defaulters_detail_rec_enq_df.repartition(1).write \
.format("csv") \
.option("header",True) \
.mode('overwrite') \
.option("path","/user/itv009959/lendingclubproject/bad/bad_data_loans_defaulters_detail_rec_enq_csv") \
.save()

In [43]:
bad_customer_consolidated_df = bad_data_customers.select("member_id") \
.union(bad_data_loans_defaulters_delinq.select("member_id")) \
.union(bad_data_loans_defaulters_detail_rec_enq_df.select("member_id"))

In [41]:
bad_customer_consolidated_df_final = bad_customer_consolidated_df.distinct()

In [24]:
bad_customer_consolidated_df_final.count()

3189

In [25]:
bad_customer_consolidated_df_final.repartition(1).write \
.format("csv") \
.option("header",True) \
.mode('overwrite') \
.option("path","/user/itv009959/lendingclubproject/bad/bad_data_customer_consolidated_finalq_csv") \
.save()

In [44]:
bad_customer_consolidated_df_final.createOrReplaceTempView("bad_data_customer")

In [27]:
customers_df = spark.sql("""
select * from itv009959_lending_club.customers where 
member_id not in (
select member_id from bad_data_customer
)
""")

In [28]:
customers_df.count()

2254192

In [29]:
customers_df.write \
.format("parquet") \
.mode("overwrite") \
.option("path","/user/itv009959/lendingclubproject/cleaned_new/customers_parquet") \
.save()

In [30]:
loans_defaulters_delinq_df = spark.sql("""
select * from itv009959_lending_club.loans_defaulters_delinq where 
member_id not in (
select member_id from bad_data_customer
)
""")

In [31]:
loans_defaulters_delinq_df.write \
.format("parquet") \
.mode("overwrite") \
.option("path","/user/itv009959/lendingclubproject/cleaned_new/loans_defaulters_delinq_parquet") \
.save()

In [45]:
loans_defaulters_delinq_detail_records_df = spark.sql("""
select * from itv009959_lending_club.loans_defaulters_delinq_detail_records where 
member_id not in (
select member_id from bad_data_customer
)
""")

In [46]:
loans_defaulters_delinq_detail_records_df.write \
.format("parquet") \
.mode("overwrite") \
.option("path","/user/itv009959/lendingclubproject/cleaned_new/loans_defaulters_delinq_detail_records_parquet") \
.save()

In [52]:
spark.sql("""
create EXTERNAL TABLE itv009959_lending_club.customers_new(
member_id string, emp_title string, emp_length int,
home_ownership string, annual_income float, address_state string, address_zipcode string, 
address_country string, grade string, sub_grade string, verification_status string, 
total_high_credit_limit float, application_type string, join_annual_income float, verification_status_joint string, ingest_date timestamp 
) stored as parquet LOCATION '/user/itv009959/lendingclubproject/cleaned_new/customers_parquet'

""")

""


In [51]:
spark.sql("drop table itv009959_lending_club.customers")

""


In [53]:
spark.sql("""
create  EXTERNAL TABLE itv009959_lending_club.loans_defaulters_delinq_new(
member_id string, dealing_2yrs int, dealing_amnt float, months_since_last_dealing int
) stored as parquet LOCATION '/user/itv009959/lendingclubproject/cleaned_new/loans_defaulters_delinq_parquet'

""")

""


In [54]:
spark.sql("""
create  EXTERNAL TABLE itv009959_lending_club.loans_defaulters_delinq_detail_records_new(
member_id string, pub_rec int, pub_rec_bankruptcies int, inquiry_last_6months int
) stored as parquet LOCATION '/user/itv009959/lendingclubproject/cleaned_new/loans_defaulters_delinq_detail_records_parquet'

""")

""


In [55]:
spark.sql(" select member_id, count(*) as total from itv009959_lending_club.customers_new group by member_id order by total desc")

Py4JJavaError: An error occurred while calling o462.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: ResultStage 11 (showString at NativeMethodAccessorImpl.java:0) has failed the maximum allowable number of times: 4. Most recent failure reason: org.apache.spark.shuffle.FetchFailedException: java.lang.IllegalArgumentException: Unknown message type: 9 	at org.apache.spark.network.shuffle.protocol.BlockTransferMessage$Decoder.fromByteBuffer(BlockTransferMessage.java:71) 	at org.apache.spark.network.shuffle.ExternalShuffleBlockHandler.receive(ExternalShuffleBlockHandler.java:80) 	at org.apache.spark.network.server.TransportRequestHandler.processRpcRequest(TransportRequestHandler.java:180) 	at org.apache.spark.network.server.TransportRequestHandler.handle(TransportRequestHandler.java:103) 	at org.apache.spark.network.server.TransportChannelHandler.channelRead(TransportChannelHandler.java:118) 	at org.spark_project.io.netty.channel.AbstractChannelHandlerContex

member_id,total
7896e26593d77002d...,1
2d812c03838313aa2...,1
8377e79819ae78fa8...,1
f50d2dbad856b169b...,1
2147c673ab2bf6757...,1
aecae2e14dcfc9340...,1
6498c88718e16a462...,1
ffeb15188e88cd928...,1
3791c7cfc042adb79...,1
57ff23ca8b9e7f61e...,1
